# <h2 align=center> PROYECTO INDIVIDUAL Nº2 </h2>
# <h1 align=center>Cryptocurrency Market Data Analytics</h1>
        @autor: Lesmen Gerardo García Pernia

#### Objetivo:<br> Realizar un análisis en el mercado de Criptomonedas para identificar oportunidades de inversión a Mediano Plazo,<Br> utilizando datos de la API CoinGecko.
### Objetivo Específicos:<Br>
1. Identificar monedas sobrecompradas o sobrevendidas en un período de tiempo específico.<br>
2. Realizar seguimiento de cambios en la fuerza de la tendendia de las monedas. <br>
3. Identificar cruces de linea de señal para identificar cambios potenciales en la tendencia de las monedas.<br>
4. Analizar la divergencia entre el MACD y el precio de las monedas. <br>
5. Evaluar el porcentaje de retorno promedio de las monedas en un período de tiempo específico. <br>  

### Versiones de las librerias utilizadas<Br>
+   pandas==1.3.5<Br>
+   numpy==1.24.3<Br>
+   pip==23.2.1<Br>
+   typing-extensions==4.5.0<Br>
+   uvicorn==0.20.0<Br>
+   scikit-learn==1.0.2<Br>
+   regex==2023.5.5<Br>
+   matplotlib==3.7.1<Br>
+   seaborn=0.12.2<Br>
+   requests==2.31.0<Br>
+   pycoingecko==3.1.0<Br>

Importando Librerias

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timedelta
import random
import pandas_ta as ta

# 1) Extracción Transformación y Carga (ETL)

### Fuente de datos:

+ [API CoinGecko](https://www.coingecko.com/es/api/documentation): Sitio Web oficial de CoinGecko para el acceso a información de Criptomonedas a través de su API.


<br/>

Creando una instancia de la libreria CoinGecko

In [2]:
cg = CoinGeckoAPI()

### 1.1. Selección de las Monedas.

Se estarán seleccionando primeramente el 10% de las Criptomonedas existente en CoinGecko (más de 10.012 monedas), bajo el criterio principal de mayor Capitalización de Mercado y luego bajo el criterio secundario de mayor Volumen de Negociación.<Br>
Luego de las primeras 200 Criptomonedas ordenadas por ambos criterios; se procede a seleccionar aleatoriamente, las 100 Criptomonedas para el análisis respectivo del presente proyecto. Y por ultimo, nos quedaremos con las 10 primeras. 

1. Total de Monedas en CoinGecko

In [3]:
### Verificando la cantidad total de Criptomonedas existentes en CoinGecko.

# Obtener la lista de criptomonedas
lista_Criptomonedas = cg.get_coins_list()

# Contar la cantidad total de criptomonedas
Total_lista_Criptomonedas = len(lista_Criptomonedas)

# Imprimir la cantidad total de Criptomonedas
print(f"La cantidad total de Criptomonedas en la API de CoinGecko es: {Total_lista_Criptomonedas}")

La cantidad total de Criptomonedas en la API de CoinGecko es: 10214


2. Porcentaje de Monedas a seleccionar Inicialmente desde CoinGecko (10 %)

In [4]:
### Porcentaje a seleccionar de las Criptomonedas
#=====================================
PorcentajeMonedasSeleccionar = 10     ### Valor definido por el Proyecto y/o Cliente.
#=====================================

# Total de Monedas a seleccionar
Total_Monedas =  round(Total_lista_Criptomonedas * PorcentajeMonedasSeleccionar / 100)

print(f"Porcentaje de Monedas a Seleccionar: {PorcentajeMonedasSeleccionar} %")
print(f"Total de Monedas a Seleccionar: {Total_Monedas}")
print(f"Se estarán seleccionando primeramente {Total_Monedas} Monedas, de un total de {Total_lista_Criptomonedas}, \nbajo el criterio inicial de mayor Capitalización de Mercado y luego bajo el criterio secundario de mayor Volumen de Negociación.")

Porcentaje de Monedas a Seleccionar: 10 %
Total de Monedas a Seleccionar: 1021
Se estarán seleccionando primeramente 1021 Monedas, de un total de 10214, 
bajo el criterio inicial de mayor Capitalización de Mercado y luego bajo el criterio secundario de mayor Volumen de Negociación.


3. Listado de Criptomonedas Ordenadas por Capitalización de Mercado.

In [6]:
### Se ordenar las monedas por capitalización de mercado. 
MonedasOrdenadas = cg.get_coins_markets(vs_currency='usd', order='market_cap_desc', per_page=Total_Monedas)

4. Listado de Criptomonedas Ordenadas por Volumen de Negociación.

In [8]:
### Luego se ordenar las monedas seleccionadas en función del Volumen de Negociación y seleccionamos solo las 200 primeras Monedas.
MonedasOrdenadas.sort(key=lambda coin: coin['total_volume'], reverse=True)
MonedasOrdenadas = MonedasOrdenadas[:200]

5. Seleccion de Monedas (100), para el Análisis.

In [9]:
# Número de monedas a seleccionar aleatoriamente ('N')
#========================================
N = 100                                    ### Total de Monedas a Seleccionar Aleatoriamente.
#========================================

# Seleccionar "N" monedas aleatoriamente de las 100 previamente seleccionadas
monedasAleatorias = random.sample(MonedasOrdenadas, N)

# Obtener los IDs de las monedas seleccionadas
ListaMonedas = [coin['id'] for coin in monedasAleatorias]
Monedas = {'Criptomonedas':ListaMonedas}
Monedas_Seleccionadas_df = pd.DataFrame(Monedas) 

# Guardando las Monedas Seleccionadas en el DataSets para poder ser leidas desde el EDA.
Monedas_Seleccionadas_df.to_csv('../DataSets/Monedas_Seleccionadas_df.csv', index=False)

print("IDs de las 100 monedas finalmente seleccionadas:")
print(ListaMonedas)

### Convertir Lista de Monedas a Dataframe
df = pd.DataFrame(monedasAleatorias)

df[['id','name','current_price','market_cap','total_volume']]


IDs de las 100 monedas finalmente seleccionadas:
['gmx', 'gnosis', 'gala', 'harmony', 'everscale', 'lido-staked-sol', 'ecomi', 'mask-network', 'first-digital-usd', 'pepe', 'ssv-network', 'aptos', 'apenft', 'flow', 'open-exchange-token', 'chainlink', 'uma', 'rocket-pool-eth', 'quant-network', 'yearn-finance', 'siacoin', 'monero', 'blockstack', 'gains-network', 'rollbit-coin', 'hedera-hashgraph', 'icon', 'zilliqa', 'litecoin', 'usdd', 'deso', 'dai', 'ontology', 'paxos-standard', 'swissborg', 'filecoin', 'iota', 'kadena', 'apecoin', 'ethereum-classic', 'aave', 'illuvium', 'usd-coin', 'cartesi', 'liquity-usd', 'bitget-token', 'neo', 'iostoken', 'arweave', 'havven', 'loopring', 'reserve-rights-token', '1inch', 'celo', 'qtum', 'nem', 'the-sandbox', 'kaspa', 'avalanche-2', 'terra-luna', 'zelcash', 'singularitynet', 'coredaoorg', 'bitcoin', 'the-open-network', 'aragon', 'enjincoin', 'radix', 'ocean-protocol', 'chia', 'thorchain', 'bitcoin-cash', 'theta-token', 'astar', 'convex-finance', 'bitco

,id,name,current_price,market_cap,total_volume
0,gmx,GMX,36.670000,329034288,9052370
1,gnosis,Gnosis,100.450000,259512909,1097621
2,gala,GALA,0.019196,492611301,508441627
3,harmony,Harmony,0.009947,122342865,8215511
4,everscale,Everscale,0.052540,94690938,3158210
...,...,...,...,...,...
95,binance-usd,Binance USD,0.999226,3148270083,2565184079
96,chiliz,Chiliz,0.062648,437158297,21968699
97,optimism,Optimism,1.500000,1077775528,137630909
98,sushi,Sushi,0.575555,111065262,15659196


### 1.2. Carga del Historial (3 años) de las Metricas de las Monedas Seleccionadas.

1.  Obteniendo el Historial de la Métrica: Precio Diario

In [12]:
### Obteniendo el Historial de la Métrica: Precio Diario.

### Crear una instancia de CoinGeckoAPI
cg = CoinGeckoAPI()

# Obtener datos históricos de 365 días de CoinGecko para las criptomonedas seleccionadas
historical_data = {}
for coin_id in ListaMonedas:
    coin_data = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency='usd', from_timestamp=int((datetime.now() - timedelta(days=365*3)).timestamp()), to_timestamp=int(datetime.now().timestamp()))
    historical_data[coin_id] = coin_data

# Crear un DataFrame a partir de los datos históricos
dataframes = []
for coin_id, data in historical_data.items():
    df = pd.DataFrame(data['prices'], columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    
    # Agregar columnas de ID, nombre y símbolo
    coin_info = cg.get_coin_by_id(coin_id)
    df["id"] = coin_id
    df["name"] = coin_info["name"]
    df["symbol"] = coin_info["symbol"]
    
    # Calcular valores de Open, High, Low y Close
    df['Open'] = df['price'].shift(1)  # Valor de ayer como Open de hoy
    df['High'] = df[['Open', 'price']].max(axis=1)  # Máximo entre Open y precio
    df['Low'] = df[['Open', 'price']].min(axis=1)   # Mínimo entre Open y precio
    df['Close'] = df['price']  # Precio de cierre es el propio precio
    
    dataframes.append(df)

# Combinar los DataFrames en uno solo
Precios_df2 = pd.concat(dataframes)

# Guardar el DataFrame como archivo CSV para ser leido desde el EDA
#Precios_df2.to_csv('../DataSets/PowerBI/DatosORG_3Anos.csv', sep=',')
print(Precios_df2.columns)



Index(['price', 'id', 'name', 'symbol', 'Open', 'High', 'Low', 'Close'], dtype='object')


In [29]:
### Información del Datsets.
Precios_df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90766 entries, 2021-09-13 00:00:00 to 2023-08-25 00:00:00
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   90766 non-null  float64
 1   id      90766 non-null  object 
 2   name    90766 non-null  object 
 3   symbol  90766 non-null  object 
 4   Open    90666 non-null  float64
 5   High    90766 non-null  float64
 6   Low     90766 non-null  float64
 7   Close   90766 non-null  float64
dtypes: float64(5), object(3)
memory usage: 6.2+ MB


2.  Obteniendo el Historial de la Métrica: Capitalización de Mercado Diario

In [21]:
### Obteniendo el Historial de la Métrica: Capitalización de Mercadp Diario.

# Crear una instancia de CoinGeckoAPI
cg = CoinGeckoAPI()

# Obtener datos históricos de 365 días de CoinGecko para las criptomonedas seleccionadas
historical_data = {}
for coin_id in ListaMonedas:
    coin_data = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency='usd', from_timestamp=int((datetime.now() - timedelta(days=365*3)).timestamp()), to_timestamp=int(datetime.now().timestamp()))
    historical_data[coin_id] = coin_data

# Crear un DataFrame a partir de los datos históricos
dataframes = []
for coin_id, data in historical_data.items():
    df = pd.DataFrame(data['market_caps'], columns=["timestamp", "market_cap"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    
    # Agregar columnas de ID, nombre y símbolo
    coin_info = cg.get_coin_by_id(coin_id)
    df["id"] = coin_id
    df["name"] = coin_info["name"]
    df["symbol"] = coin_info["symbol"]
    
    dataframes.append(df)

# Combinar los DataFrames en uno solo
CapitalizacionMercados_df = pd.concat(dataframes)

# Guardar el DataFrame como archivo CSV para ser leido desde el EDA
#CapitalizacionMercados_df.to_csv('../DataSets/CapitalizacionMercados_df.csv', sep=',')
print(CapitalizacionMercados_df.columns)

Index(['market_cap', 'id', 'name', 'symbol'], dtype='object')


In [28]:
### Información del Datsets.
CapitalizacionMercados_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90766 entries, 2021-09-13 00:00:00 to 2023-08-25 00:00:00
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   market_cap  90766 non-null  float64
 1   id          90766 non-null  object 
 2   name        90766 non-null  object 
 3   symbol      90766 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.5+ MB


3.  Obteniendo el Historial de la Métrica: Volumen de Negociación Diario.

In [24]:
### Obteniendo el Historial de la Métrica: Volumen de Negociación Diario.

# Crear una instancia de CoinGeckoAPI
cg = CoinGeckoAPI()

# Obtener datos históricos de 365 días de CoinGecko para las criptomonedas seleccionadas
historical_data = {}
for coin_id in ListaMonedas:
    coin_data = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency='usd', from_timestamp=int((datetime.now() - timedelta(days=365*3)).timestamp()), to_timestamp=int(datetime.now().timestamp()))
    historical_data[coin_id] = coin_data

# Crear un DataFrame a partir de los datos históricos
dataframes = []
for coin_id, data in historical_data.items():
    df = pd.DataFrame(data['total_volumes'], columns=["timestamp", "total_volumen"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    
    # Agregar columnas de ID, nombre y símbolo
    coin_info = cg.get_coin_by_id(coin_id)
    df["id"] = coin_id
    df["name"] = coin_info["name"]
    df["symbol"] = coin_info["symbol"]
    
    dataframes.append(df)

# Combinar los DataFrames en uno solo
VolumenNegociacion_df = pd.concat(dataframes)

# Guardar el DataFrame como archivo CSV para ser leido desde el EDA
VolumenNegociacion_df.to_csv('../DataSets/VolumenNegociacion_df.csv', sep=',')
print(VolumenNegociacion_df.columns)

Index(['total_volumen', 'id', 'name', 'symbol'], dtype='object')


In [27]:
### Información del Datsets.
VolumenNegociacion_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90766 entries, 2021-09-13 00:00:00 to 2023-08-25 00:00:00
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   total_volumen  90766 non-null  float64
 1   id             90766 non-null  object 
 2   name           90766 non-null  object 
 3   symbol         90766 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.5+ MB


### 1.3.  Union de los 3 DataSets en uno Solo, llamado: 'DatosTotal_df'

In [32]:
DatosCriptos = Precios_df2.merge(CapitalizacionMercados_df, on=['id', 'name', 'symbol', 'timestamp'])
DatosCriptos = DatosCriptos.merge(VolumenNegociacion_df, on=['id', 'name', 'symbol', 'timestamp'])

# Resetea los índices
#DatosTotal_df = DatosTotal_df.reset_index(drop=True)

# Guardar el DataFrame DatosTotal_df como archivo CSV para ser leido desde el EDA
DatosCriptos.to_csv('../DataSets/DatosCriptos.csv',  index=True, sep=',')

# Muestra el DataFrame combinado con el nuevo índice 'timestamp'
DatosCriptos

,price,id,name,symbol,Open,High,Low,Close,market_cap,total_volumen
timestamp,,,,,,,,,,
2021-09-13,15.732550,gmx,GMX,gmx,NaN,15.732550,15.732550,15.732550,0.000000e+00,1.297668e+05
2021-09-14,15.732550,gmx,GMX,gmx,15.732550,15.732550,15.732550,15.732550,0.000000e+00,1.297668e+05
2021-09-15,21.062166,gmx,GMX,gmx,15.732550,21.062166,15.732550,21.062166,0.000000e+00,2.164767e+05
2021-09-16,22.646664,gmx,GMX,gmx,21.062166,22.646664,21.062166,22.646664,0.000000e+00,4.623126e+04
2021-09-17,20.643008,gmx,GMX,gmx,22.646664,22.646664,20.643008,20.643008,0.000000e+00,8.569191e+04
...,...,...,...,...,...,...,...,...,...,...
2023-08-21,2.238064,tominet,tomiNet,tomi,2.167387,2.238064,2.167387,2.238064,1.665534e+08,1.902348e+07
2023-08-22,2.270774,tominet,tomiNet,tomi,2.238064,2.270774,2.238064,2.270774,1.691172e+08,1.769247e+07
2023-08-23,2.241901,tominet,tomiNet,tomi,2.270774,2.270774,2.241901,2.241901,1.672254e+08,1.868110e+07


In [33]:
DatosCriptos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90766 entries, 2021-09-13 00:00:00 to 2023-08-25 00:00:00
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          90766 non-null  float64
 1   id             90766 non-null  object 
 2   name           90766 non-null  object 
 3   symbol         90766 non-null  object 
 4   Open           90666 non-null  float64
 5   High           90766 non-null  float64
 6   Low            90766 non-null  float64
 7   Close          90766 non-null  float64
 8   market_cap     90766 non-null  float64
 9   total_volumen  90766 non-null  float64
dtypes: float64(7), object(3)
memory usage: 7.6+ MB


Informacion del DataFrame (DatosCriptos) con los Datos Históricos

In [35]:
### Cantidad de Columnas del DataFrame
print("Cantidad de Columnas del DataFrame Historico: ", len(DatosCriptos.columns))
print("Columnas del DataFrame Historico: ", DatosCriptos.columns)
### Dimensiones de DataFrame
print("Dimensiones del DataFrame Historico: ", DatosCriptos.shape)

Cantidad de Columnas del DataFrame Historico:  10
Columnas del DataFrame Historico:  Index(['price', 'id', 'name', 'symbol', 'Open', 'High', 'Low', 'Close',
       'market_cap', 'total_volumen'],
      dtype='object')
Dimensiones del DataFrame Historico:  (90766, 10)


Nota: El Dataframe tiene el 'timestamp' como indice. Pero al cargarlos en el EDA, se visualizará como otra columnas adiciona. <br> 
Dicha columna, será configurada en muchos casos, como índice principal, ya que estaremos trabajando con Series de Tiempo; que es la naturaleza principal del presente Proyecto de CriptoMonedas.

### Fin del ETC

## 2. Analisis Exploratorios de los Datos (EDA)

El archivo del EDA (EDA_Criptomonedas.ipynb) se encuentra en la carpeta de nombre 'EDA'.